In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
# model_checkpoint = "lmsys/vicuna-13b-v1.5" # 노트북 로컬에서는 메모리가 안된다!ㅠㅠ
model_checkpoint = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(tokenizer.eos_token)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, pad_token_id=tokenizer.eos_token_id)

<|endoftext|>


In [91]:
input = "I want to compose a story about Hamas and Israel War."
history = []

In [92]:
# def predict(input, history=[]):
print(input)
new_user_input_ids = tokenizer.encode(
    input+tokenizer.eos_token, return_tensors="pt"
)
print(new_user_input_ids)

bot_input_ids = torch.cat([torch.LongTensor(history), new_user_input_ids], dim=-1)
print(bot_input_ids)

I want to compose a story about Hamas and Israel War.
tensor([[   40,   765,   284, 36664,   257,  1621,   546, 15767,   290,  2692,
          1810,    13, 50256]])
tensor([[   40,   765,   284, 36664,   257,  1621,   546, 15767,   290,  2692,
          1810,    13, 50256]])


In [93]:
history = model.generate(
    bot_input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id
).tolist()

A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


In [94]:
# \n 가 연속되는 경우는 1개의 \n으로 바꿈
new_history = []
prev_tok = ''
for tok_idx in history[0]:
    if tok_idx == prev_tok == newline_token_id:
        continue
    else:
        new_history.append(tok_idx)
        prev_tok = tok_idx
if new_history[-1] != tokenizer.eos_token_id and new_history[-1] == newline_token_id:
    new_history[-1] = tokenizer.eos_token_id
elif new_history[-1] != tokenizer.eos_token_id:
    new_history.append(tokenizer.eos_token_id)

In [95]:
history = [new_history]

In [96]:
response = tokenizer.decode(history[0]).split("<|endoftext|>")
response = [
    (response[i], response[i+1]) for i in range(0, len(response) - 1, 2)
]
    
    # return response, history

In [97]:
response

[('I want to compose a story about Hamas and Israel War.',
  'The first of the three-part series of articles on the Israeli-Palestinian conflict in the Middle East is a series of articles on the Israeli-Palestinian conflict in the Middle East.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian conflict in the Middle East is a conflict between Israel and the Palestinians.\nThe Israeli-Palestinian con